<a href="https://colab.research.google.com/github/Bhavya171/LLM_From_Scratch/blob/main/Chapter_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 2

Loading the-verdict

In [ ]:
with open("/content/drive/MyDrive/MAT496/the-verdict.txt","r",encoding = "utf-8") as f:
  raw_text = f.read()
print("Total number opf character:", len(raw_text))
print(raw_text[:99])

Total number opf character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [ ]:
# separating words and spaces

import re
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
print(result)

# separating the punctuations

result = re.split(r'([,.]|\s)', text)
print(result)

result = [item for item in result if item.strip()]
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']
['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']
['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [ ]:
# applying this to the full story

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print("Total nos of tokens:",len(preprocessed))

print(preprocessed[:30])

Total nos of tokens: 4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [ ]:
# creating a list of all tokens alphabetically

all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [ ]:
# creating a vocabulary out of it

vocab = {token: integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
  print(item)
  if i >= 50: # printing the first 50 words
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


## Simple Program to encode/decode

In [ ]:
# making a simple program to encode and then to decode the text

# the program can encode the text to tokens and then also decode from the token ids into retrieving the text

class SimpleTokenizerV1:
  def __init__(self,vocab):
    self.str_to_int = vocab
    self.int_to_str = {i:s for s,i in vocab.items()}

  def encode(self,text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
    preprocessed = [
        item.strip() for item in preprocessed if item.strip()
    ]

    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self,ids):
    text = " ".join([self.int_to_str[i] for i in ids])

    text =  re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text

In [ ]:
# using this

tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
 Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [ ]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [ ]:
# trying it on testing data which will give an error beacause the word 'Hello' is not there in the training dataset

text = " do you like tea?"
print(tokenizer.encode(text))

[355, 1126, 628, 975, 10]


----

In [ ]:
# modyfying the model to add 2 new words <|unk|> and <|endoftext|>

all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}

print(len(vocab.items()))

1132


In [ ]:
for i, item in enumerate(list(vocab.items())[-5:]):
 print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [ ]:
# updating out model to handle these new words as well


class SimpleTokenizerV2:
  def __init__(self,vocab):
    self.str_to_int = vocab
    self.int_to_str = {i:s for s,i in vocab.items()}

  def encode(self,text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
    preprocessed = [
        item.strip() for item in preprocessed if item.strip()
    ]

    preprocessed = [item if item in self.str_to_int
                    else "<|unk|>" for item in preprocessed]

    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self,ids):
    text = " ".join([self.int_to_str[i] for i in ids])

    text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
    return text

In [ ]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palaces"
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palaces


In [ ]:
tokenizer = SimpleTokenizerV2(vocab)

print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))


[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>


## Byte Pair Encoding (BPE)

In [ ]:
!pip install tiktoken

In [ ]:
import tiktoken

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
integers = tokenizer.encode(text,allowed_special = {"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 262, 6340, 2114]


In [ ]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palaces


In [ ]:
text = "I am Bhavya Jain"
num = tokenizer.encode(text)
print(num)

[40, 716, 16581, 2830, 64, 449, 391]


In [ ]:
print(tokenizer.decode(num))

I am Bhavya Jain


In [ ]:
# encoding verdict.txt with tokenizer

with open("/content/drive/MyDrive/MAT496/the-verdict.txt","r",encoding = "utf-8") as f:
  raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [ ]:
enc_sample = enc_text[50:]

In [ ]:
# using the sliding window technique to guess the next word up

context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [ ]:
for i in range(1,context_size+1):
  input = enc_sample[:i]
  output = enc_sample[i]
  print(input,"------",output)

[290] ------ 4920
[290, 4920] ------ 2241
[290, 4920, 2241] ------ 287
[290, 4920, 2241, 287] ------ 257


In [ ]:
for i in range(1,context_size+1):
  input = enc_sample[:i]
  output = enc_sample[i]
  print(f"{tokenizer.decode(input)}---{tokenizer.decode([output])})")

 and--- established)
 and established--- himself)
 and established himself--- in)
 and established himself in--- a)


## Making GPTDatasetV1

In [ ]:
# making tensors for input and output but the output should be moved by 1 place

import torch
import tiktoken
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
  def __init__(self,txt,tokenizer,max_length,stride):
    self.input_ids = []
    self.output_ids = []

    token_ids = tokenizer.encode(txt)

    for i in range(0,len(token_ids) - max_length, stride):
        input_chunk = token_ids[i:i+max_length]
        output_chunk = token_ids[i+1:i+max_length+1]
        self.input_ids.append(torch.tensor(input_chunk))
        self.output_ids.append(torch.tensor(output_chunk))

  def __len__(self):
      return len(self.input_ids)

  def __getitem__(self,idx):
      return self.input_ids[idx], self.output_ids[idx]



In [ ]:
txt_as_dataset = GPTDatasetV1(raw_text, tokenizer, 7, 3)
txt_as_dataset.input_ids[1]

tensor([ 1464,  1807,  3619,   402,   271, 10899,  2138])

In [ ]:
txt_as_dataset.output_ids[1]

tensor([ 1807,  3619,   402,   271, 10899,  2138,   257])

In [ ]:
# creating the dataloader for the same

def DataLoaderV1(txt, batch_size=4, max_length=256,
                  stride=128, shuffle=True, drop_last=True, num_workers=0):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(txt,tokenizer,max_length,stride)
  dataloader = DataLoader(
      dataset,
      batch_size= batch_size,
      num_workers=num_workers,
      shuffle = shuffle,
      drop_last = drop_last

  )

  return dataloader

In [ ]:
with open("/content/drive/MyDrive/MAT496/the-verdict.txt", "r", encoding = "utf-8") as f:
  raw_text = f.read()

dataloader = DataLoaderV1(raw_text, batch_size=1, max_length=4,stride = 4,shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [ ]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [ ]:
a = GPTDatasetV1(raw_text,tokenizer,8,4)
print(a.input_ids[0],a.output_ids[0],a.input_ids[1])

tensor([  40,  367, 2885, 1464, 1807, 3619,  402,  271]) tensor([  367,  2885,  1464,  1807,  3619,   402,   271, 10899]) tensor([ 1807,  3619,   402,   271, 10899,  2138,   257,  7026])


In [ ]:
dataloader = DataLoaderV1(
    raw_text, batch_size = 8, max_length = 4, stride = 3, shuffle = False
)


In [ ]:
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(inputs,targets)

tensor([[   40,   367,  2885,  1464],
        [ 1464,  1807,  3619,   402],
        [  402,   271, 10899,  2138],
        [ 2138,   257,  7026, 15632],
        [15632,   438,  2016,   257],
        [  257,   922,  5891,  1576],
        [ 1576,   438,   568,   340],
        [  340,   373,   645,  1049]]) tensor([[  367,  2885,  1464,  1807],
        [ 1807,  3619,   402,   271],
        [  271, 10899,  2138,   257],
        [  257,  7026, 15632,   438],
        [  438,  2016,   257,   922],
        [  922,  5891,  1576,   438],
        [  438,   568,   340,   373],
        [  373,   645,  1049,  5975]])


## Creating token embeddings

In [ ]:
vocab_size = 10
output_dim = 5

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size,output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  0.3486],
        [ 0.6603, -0.2196, -0.3792,  0.7671, -1.1925],
        [ 0.6984, -1.4097,  0.1794,  1.8951,  0.4954],
        [ 0.2692, -0.0770, -1.0205, -0.1690,  0.9178],
        [ 1.5810,  1.3010,  1.2753, -0.2010,  0.4965],
        [-1.5723,  0.9666, -1.1481, -1.1589,  0.3255],
        [-0.6315, -2.8400, -1.3250,  0.1784, -1.4779],
        [ 1.1331, -1.2203,  1.3139,  1.0533,  0.1388],
        [-0.2044, -2.2685, -0.2808,  0.7697, -0.6596],
        [-0.7979,  0.1838,  0.2293,  0.6177, -0.2876]], requires_grad=True)


In [ ]:
print(embedding_layer(torch.tensor([3])))

tensor([[ 0.2692, -0.0770, -1.0205, -0.1690,  0.9178]],
       grad_fn=<EmbeddingBackward0>)


In [ ]:
input_ids = torch.tensor([6, 1])
embedding_layer(input_ids)

tensor([[-0.6315, -2.8400, -1.3250,  0.1784, -1.4779],
        [ 0.6603, -0.2196, -0.3792,  0.7671, -1.1925]],
       grad_fn=<EmbeddingBackward0>)